In [1]:
%pip install wikipedia

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [1]:
import os
import requests

import wikipedia 

from bs4 import BeautifulSoup

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

In [2]:
os.environ["OPENAI_API_KEY"] = "***"

In [3]:
TREND_LOC = "FR"

In [4]:
def get_trends(locale):
    url = f"https://trends.google.fr/trends/trendingsearches/daily/rss?geo={locale}"
    trend_response = requests.get(url=url)
    soup = BeautifulSoup(trend_response.text, "xml")

    items = soup.findAll("item")
    return [item.title.text for item in items]

In [5]:
def get_wiki(topic, n=5, loc="fr"):
    wikipedia.set_lang(loc)
    return topic+":"+",".join(wikipedia.search(topic, results = n))

In [6]:
trends = get_trends('FR')
contexts = [get_wiki(trend, n=5, loc="fr") for trend in trends]

In [13]:
template = """<context>
{context}
</context>
Based on the above context (delimited by <context> tags)
Generate multiple question respecting the below template:

"What are the top 6 ... this week?"

The answer to this question must include at least two elements from the context (delimited by <context> tags):
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context"],
)

model = ChatOpenAI()
chain = prompt | model

context = "\n".join(contexts)
llm_output = chain.invoke({"context": context})
print(llm_output.content)

1. What are the top 6 Celine Dion songs this week?
The answer could include "The Power of Love" and "I'm Alive" as two of the top songs by Celine Dion.

2. What are the top 6 Grammy Awards records this week?
The answer could include the "Grammy Award for Song of the Year" and the "Grammy Award for Album of the Year" as two of the top records at the Grammy Awards.

3. What are the top 6 football clubs in Portugal this week?
The answer could include Benfica Lisbonne and Gil Vicente Futebol Clube as two of the top football clubs in Portugal.

4. What are the top 6 French judo athletes this week?
The answer could include Teddy Riner and Clarisse Agbégnénou as two of the top French judo athletes.

5. What are the top 6 Taylor Swift albums this week?
The answer could include "Taylor Swift" and "Evermore" as two of the top albums by Taylor Swift.

6. What are the top 6 fashion designers this week?
The answer could include Simon Porte Jacquemus and Úrsula Corberó as two of the top fashion desi